### 1. SSH to the device and execute 'show hostname' command:

In [1]:
from netmiko import ConnectHandler

In [2]:
try:
    device_connection = ConnectHandler(
        device_type = 'cisco_nxos',
        ip = '127.0.0.1',
        port = '2222',
        username = 'admin',
        password = 'Cisco!123'
    )
    print("Connected to the device!")
except:
    print("Failure...")

Connected to the device!


In [3]:
ssh_output = device_connection.send_command("show hostname")

In [4]:
print(ssh_output)

Nexus9000v 



In [5]:
device_connection.disconnect()

### 2. Create NETCONF Request that will GET the Hostname of your device:

* **ncclient** - Python library that facilitates scripting around the NETCONF protocol

In [1]:
import ncclient
from ncclient import manager
from ncclient.operations import TimeoutExpiredError
import xml.dom.minidom

In [4]:
try:
    device_connection = ncclient.manager.connect(
        host = '127.0.0.1',
        port = 2222,
        username='admin',
        password='Cisco!123',
        hostkey_verify=False,
        device_params={'name': 'nexus'},
        allow_agent=False,
        look_for_keys=False
    )
    print("Connected to the device!")
except:
    print("Failure...")

Connected to the device!


In [5]:
int_filter = '''
                          <show xmlns="http://www.cisco.com/nxos:1.0">
                              <hostname>
                              </hostname>
                          </show>
             '''

And print the received **NETCONF Response** in an original **XML** format:

In [6]:
netconf_output = device_connection.get(('subtree', int_filter))
print(netconf_output)

<?xml version="1.0" encoding="ISO-8859-1"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns:mod="http://www.cisco.com/nxos:1.0:vdc_mgr" message-id="urn:uuid:58b54e61-7593-4020-973b-5f157269594f" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <mod:show>
   <mod:__XML__BLK_Cmd_SHOW_HOSTNAME_hostname>
    <mod:__XML__OPT_Cmd_SHOW_HOSTNAME___readonly__>
     <mod:__readonly__>
      <mod:hostname>Nexus9000v</mod:hostname>
     </mod:__readonly__>
    </mod:__XML__OPT_Cmd_SHOW_HOSTNAME___readonly__>
   </mod:__XML__BLK_Cmd_SHOW_HOSTNAME_hostname>
  </mod:show>
 </data>
</rpc-reply>


 **Pass the XML result as a string to the parseString function**

In [7]:
        xml_doc = xml.dom.minidom.parseString(netconf_output.xml)
        hostname = xml_doc.getElementsByTagName("mod:hostname")
        print(hostname[0].firstChild.nodeValue)

Nexus9000v


### 3. Create NETCONF Request that will change the Hostname of your device to 'NEXUS':

In [23]:
update_hostname = '''
            <configure xmlns="http://www.cisco.com/nxos:1.0">
                <__XML__MODE__exec_configure>
                    <hostname><name>%s</name></hostname>
                </__XML__MODE__exec_configure>   
            </configure>
'''


In [24]:
def update_config():
    try:
        device_connection = ncclient.manager.connect(
            host = '127.0.0.1',
            port = 2222,
            username='admin',
            password='Cisco!123',
            hostkey_verify=False,
            device_params={'name': 'nexus'},
            allow_agent=False,
            look_for_keys=False
        )
        print("Connected to the device!")
    except:
        print("Failure...")
        
    configuration = ''

    try:
        hostname = "NEXUS"
        configuration += '<config>'
        configuration += update_hostname %hostname
        configuration += '</config>'
        print(configuration)
        device_connection.edit_config(target='running', config=configuration)
        print("Config pushed successfuly!")
    except:
        print("Something went wrong...")

In [25]:
 update_config()

Connected to the device!
<config>
            <configure xmlns="http://www.cisco.com/nxos:1.0">
                <__XML__MODE__exec_configure>
                    <hostname><name>NEXUS</name></hostname>
                </__XML__MODE__exec_configure>   
            </configure>
</config>
Config pushed successfuly!
